In [1]:
!pip install --upgrade json-repair networkx langchain-core langchain-experimental langchain-community langchain_openai

  Using cached json_repair-0.39.0-py3-none-any.whl.metadata (11 kB)
  Using cached langchain_experimental-0.3.4-py3-none-any.whl.metadata (1.7 kB)
Using cached json_repair-0.39.0-py3-none-any.whl (20 kB)
Using cached langchain_experimental-0.3.4-py3-none-any.whl (209 kB)


In [2]:
import os
os.environ["OPENAI_API_KEY"] = "sk" # openai 키 입력, os.environ을 사용하여 환경 변수에 OpenAI API 키를 설정

from langchain_openai import ChatOpenAI
llm = ChatOpenAI(  # ChatOpenAI 클래스를 사용하여 OpenAI의 GPT-4o 모델을 호출
    model="gpt-4o",
    temperature=0,  # 모델의 출력을 일관되게 유지 (창의성을 최소화)
)

# text 변수에 마리 퀴리에 대한 설명을 저장
text = """
마리 퀴리(Marie Curie)는 1867년 태어난 폴란드 출신으로, 이후 프랑스로 귀화한 물리학자이자 화학자입니다. 그녀는 방사능에 관한 선구적인 연구를 수행하였습니다.
그녀는 노벨상을 수상한 최초의 여성, 두 번이나 노벨상을 수상한 최초의 인물, 그리고 두 개의 과학 분야에서 노벨상을 수상한 유일한 인물입니다.
첫 번째 노벨상은 남편 피에르 퀴리(Pierre Curie) 와 공동으로 수상하였으며, 이는 역사상 최초로 부부가 함께 노벨상을 수상한 사례였습니다. 이후 퀴리 가문은 총 5개의 노벨상을 배출하며 학문의 업적을 이어갔습니다.
또한, 1906년에는 파리 대학교(Université de Paris)에서 최초의 여성 교수가 되며 학계에서 또 한 번의 중요한 이정표를 세웠습니다.
"""  

In [3]:
from langchain_core.documents import Document
documents = [Document(page_content=text)] # 이전에 정의한 text (마리 퀴리에 대한 설명)를 문서 객체로 변환합니다

from langchain_experimental.graph_transformers import LLMGraphTransformer # "문서(마리 퀴리를 설명하는 text)에서 개념과 관계를 추출하여 그래프 데이터로 변환
llm_transformer = LLMGraphTransformer(llm=llm) 
graph_documents = llm_transformer.convert_to_graph_documents(documents) # 텍스트 문서를 그래프 구조로 변환, 결과는 graph_documents에 저장되며, 노드(Node)와 엣지(Edge)로 표현될 수 있습니다.

In [9]:
llm_transformer_filtered = LLMGraphTransformer(
    llm=llm,
    allowed_nodes=["Person", "Country", "Organization"],
    allowed_relationships=["NATIONALITY", "LOCATED_IN", "WORKED_AT", "SPOUSE"],
)
graph_documents_filtered = llm_transformer_filtered.convert_to_graph_documents(
    documents
) # 허용된 개념과 관계만 포함된 그래프 데이터가 생성

In [11]:
from langchain_community.graphs.networkx_graph import NetworkxEntityGraph
graph = NetworkxEntityGraph() # NetworkxEntityGraph() 객체를 생성하여 빈 그래프를 생성

# 그래프에 노드 추가
for node in graph_documents_filtered[0].nodes: # node에 LLM이 추출한 개념(노드)을 저장
    graph.add_node(node.id) # NetworkX 그래프에 노드를 추가

# 그래프에 엣지 추가
for edge in graph_documents_filtered[0].relationships:
    graph._graph.add_edge(
            edge.source.id, # 관계의 출발점
            edge.target.id, # 관계의 도착점
            relation=edge.type, # 관계의 종류
        )

In [13]:
from langchain.chains import GraphQAChain # 그래프 데이터에 대한 질의응답(QA)을 수행하는 랭체인의 체인

# LLM과 그래프를 연결하여 Q&A 체인 생성
chain = GraphQAChain.from_llm( 
    llm=llm,
    graph=graph, # NetworkXEntityGraph에서 생성한 그래프 데이터 사용
    verbose=True
)

In [15]:
question = """Marie Curie가 누구야?"""
chain.invoke(question)



> Entering new GraphQAChain chain...
Entities Extracted:
Marie Curie
Full Context:
Marie Curie NATIONALITY Poland
Marie Curie NATIONALITY France
Marie Curie SPOUSE Pierre Curie
Marie Curie WORKED_AT Université De Paris

> Finished chain.


{'query': 'Marie Curie가 누구야?',
 'result': 'Marie Curie는 폴란드와 프랑스 국적을 가진 과학자입니다. 그녀는 피에르 퀴리와 결혼했으며, 파리 대학교에서 일했습니다.'}

In [16]:
question = """Marie Curie가 태어난 곳은?"""
chain.invoke(question)



> Entering new GraphQAChain chain...
Entities Extracted:
Marie Curie
Full Context:
Marie Curie NATIONALITY Poland
Marie Curie NATIONALITY France
Marie Curie SPOUSE Pierre Curie
Marie Curie WORKED_AT Université De Paris

> Finished chain.


{'query': 'Marie Curie가 태어난 곳은?',
 'result': 'Marie Curie의 출생지는 주어진 정보에 포함되어 있지 않습니다.'}

In [17]:
question = """Marie Curie의 출신은?"""
chain.invoke(question)



> Entering new GraphQAChain chain...
Entities Extracted:
Marie Curie
Full Context:
Marie Curie NATIONALITY Poland
Marie Curie NATIONALITY France
Marie Curie SPOUSE Pierre Curie
Marie Curie WORKED_AT Université De Paris

> Finished chain.


{'query': 'Marie Curie의 출신은?', 'result': 'Marie Curie의 출신은 폴란드입니다.'}